In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from model.pytorchtools import EarlyStopping
import model.net as models 
from model.dataset import SurfaceComplexationDataset
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

In [2]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
    return optimizer, scheduler

In [3]:
def load_data(data_dir): 
    train_set = SurfaceComplexationDataset(root_dir=data_dir, split = 'train')
    test_set = SurfaceComplexationDataset(root_dir=data_dir, split='test')
    val_set = SurfaceComplexationDataset(root_dir=data_dir, split='val')

    return train_set, test_set, val_set

In [4]:
def train_epoch(train_loader, model, optimizer, device, epoch):
    """ Train the model on num_steps batches 
    Args: 
        train_loader: a torch.utils.data.DataLoader object that fetches the data
        model: the neural network 
        optimizer: adams 
    """
    model.train()
    running_loss = 0.0
    num_batch = len(train_loader)

    for i, (inputs, targets) in enumerate(train_loader): 
        inputs, targets = inputs.to(device), targets.to(device)
        # zero the paramter gradients 
        optimizer.zero_grad()

        # forward + backward + optimize 
        pred = model(inputs)
        loss = F.mse_loss(pred, targets)
        loss.backward()
        optimizer.step()

        # print statistics 
        running_loss += loss.item()
        # if i % 300 == 0: 
        #     print('[%d: %d/%d] train loss: %f ' % (epoch, i, num_batch, loss.item()))
        # if i % 300 == 0: 
        #     print('[%d: %d/%d] train loss: %f lr = %f' % (epoch, i, num_batch, loss.item(), optimizer.param_groups[0]["lr"]))

    return running_loss / num_batch 

In [5]:
def validate(val_dataloader, model, device): 
    model.eval()
    val_running_loss = 0.0 

    with torch.no_grad(): 
        for inputs, targets in val_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = F.mse_loss(outputs, targets)

            val_running_loss += loss.item() * inputs.size(0)

    return val_running_loss / len(val_dataloader.dataset)

In [6]:
def plot_pramas(test_y, test_pred, foldername, outfile): 
    # print("R2 of training is: ", r2_score(train_y, train_pred))
    np.savetxt(f'{foldername}/test_pred_{outfile}.txt', test_pred)
    np.savetxt(f'{foldername}/test_y_{outfile}.txt', test_y)
    
    print("R2 of test is: ", r2_score(test_y, test_pred[:,4:]))

    test_mse = mean_squared_error(test_y, test_pred[:,4:])
    test_mae = mean_absolute_error(test_y, test_pred[:,4:])

    print('Test set results for %i samples:' % test_pred.shape[0])
    print('MSE:', test_mse)
    print('MAE:', test_mae)

In [7]:
def test_accuracy(net, testloader, foldername, outfile, device): 
    test_pred = []
    test_y = []
    running_loss = 0 

    with torch.no_grad():
        for data in testloader:
            inputs, targets = data
            sys = inputs[:, :4]
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = F.mse_loss(outputs, targets)

            running_loss += loss.item() 

            pred_val_numpy = outputs.data.cpu().numpy()
            target_val_numpy = targets.data.cpu().numpy()

            # test_pred.append(pred_val_numpy)
            test_pred.append(np.concatenate([sys.data.cpu().numpy(),pred_val_numpy],axis = 1))
            test_y.append(target_val_numpy)

    test_pred = np.concatenate(test_pred, axis=0)
    test_y = np.concatenate(test_y, axis=0)

    plot_pramas(test_y, test_pred, foldername, outfile)

    print('MSE loss on test set is:', running_loss / len(testloader.dataset)) 

In [8]:
def train_model(model, device, train_loader, val_loader, test_loader, optimizer, lr_scheduler, isSch, res_dir, name, patience = 20, n_epochs = 100): 
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 

    blue = lambda x: '\033[94m' + x + '\033[0m'
    
    checkpoint_dir = os.path.join(res_dir, 'checkpoints')
    try:
        os.makedirs(res_dir)
        os.makedirs(checkpoint_dir)
    except OSError:
        pass

    checkpoint_path = os.path.join(checkpoint_dir, f'{name}.pt')
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True, path = checkpoint_path)

    for epoch in tqdm(range(1, n_epochs + 1)):
        ###################
        # train the model #
        ###################
        train_epoch_loss = train_epoch(train_loader, model, optimizer, device, epoch)
        val_epoch_loss = validate(val_loader, model, device)

        # print loss every epoch 
        print('[%d] train loss: %f ' % (epoch, train_epoch_loss))
        print('[%d] %s loss: %f' % (epoch, blue('validate'), val_epoch_loss))

        avg_train_losses.append(train_epoch_loss)
        avg_valid_losses.append(val_epoch_loss)
        
        if isSch: 
            lr_scheduler.step(val_epoch_loss) 
        
        # add early stopping 
        # early_stopping(val_epoch_loss, model)
        early_stopping(train_epoch_loss, model)
        if early_stopping.early_stop: 
            print("Early stopping")
            break 

    np.savetxt(os.path.join(res_dir, f'train_loss_{name}.csv'), avg_train_losses)
    np.savetxt(os.path.join(res_dir, f'val_loss_{name}.csv'), avg_valid_losses) 

    # load the last checkpoint with the best model
    model.load_state_dict(torch.load(checkpoint_path)) 

    # test on test set 
    test_accuracy(model, test_loader, res_dir, name, device)
    # print(optimizer.state_dict())

In [9]:
def train_main(config): 
    data_dir = 'dataset/'

    # get dataset 
    train_set, val_set, test_set = load_data(data_dir)

    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=4, 
        pin_memory=False)

    val_loader = torch.utils.data.DataLoader(
            val_set,
            batch_size=int(config["batch_size"]),
            shuffle=True,
            num_workers=4, 
            pin_memory=False)

    test_loader = torch.utils.data.DataLoader(
            test_set, 
            batch_size=int(config["batch_size"]), 
            shuffle=True,
            num_workers=4, 
            pin_memory=False)
            
    print("Creating model")
    Model = getattr(models, config['model'])
    print('created model is: ', Model)
    
    model = Model(config['input_dim'], config['batch_norm'], config['layer_norm'], config['constraint'],
                     config["l1"], config["l2"], config["l3"], config["l4"], config["l5"])
        
    name = "DNN"
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer, lr_scheduler = build_optimizer(model, config['optimizer'], config['lr'])
    res_dir = 'DNN_res'
    
    train_model(model, device, train_loader, val_loader, test_loader, optimizer, lr_scheduler, config['lr_scheduler'], res_dir, name, 20, 5000)

# train DNN 

In [12]:
config = {'input_dim': 664,'l1': 1024, 'l2': 1024, 'l3': 1024, 'l4': 1024, 'l5': 512, 
          'lr': 0.01, 'batch_size': 128, 'model': 'DeepNet6LayerTune', 'batch_norm': False, 
          'layer_norm': True, 'lr_scheduler': True, 'constraint': False, 'optimizer': 'adam'}

In [ ]:
import time 
start_time = time.perf_counter()
train_main(config) 
end_time = time.perf_counter() 
print('time used to train model with 40/1000 patience is: ', (end_time - start_time)/60, 'mins')

Creating model
created model is:  <class 'model.net.DeepNet6LayerTune'>


  0%|          | 1/5000 [01:13<101:25:03, 73.04s/it]

[1] train loss: 0.018467 
[1] validate loss: 0.013492
Validation loss decreased (inf --> 0.018467).  Saving model ...


  0%|          | 2/5000 [02:23<99:33:27, 71.71s/it] 

[2] train loss: 0.009112 
[2] validate loss: 0.007725
Validation loss decreased (0.018467 --> 0.009112).  Saving model ...


...

[310] train loss: 0.000861 
[310] validate loss: 0.001042
Validation loss decreased (0.000861 --> 0.000861).  Saving model ...


  6%|▌         | 311/5000 [5:36:44<80:23:23, 61.72s/it]

[311] train loss: 0.000860 
[311] validate loss: 0.001097
Validation loss decreased (0.000861 --> 0.000860).  Saving model ...


  6%|▌         | 312/5000 [5:37:45<80:13:30, 61.61s/it]

[312] train loss: 0.000860 
[312] validate loss: 0.001024
Validation loss decreased (0.000860 --> 0.000860).  Saving model ...
